In [16]:
import acnportal
import sklearn

from copy import deepcopy
import warnings
import pytz
import numpy as np
import pandas as pd
import pickle
from datetime import datetime
from acnportal import acnsim
from acnportal import algorithms
from acnportal.signals.tariffs.tou_tariff import TimeOfUseTariff
from acnportal.acnsim.events import GaussianMixtureEvents
from acnportal.contrib.acnsim import StochasticNetwork
from acnportal.acnsim.network import ChargingNetwork
from modified_adacharge import modified_adacharge

In [17]:
def ev_fleet_level_2_network(transformer_cap=30, evse_per_phase=2):
    """ Configurable charging network for level-2 EVSEs connected line to line
        at 415 V.

    Args:
        transformer_cap (float): Capacity of the transformer feeding the network
          [kW]
        evse_per_phase (int): Number of EVSEs on each phase. Total number of
          EVSEs will be 3 * evse_per_phase.

    Returns:
        ChargingNetwork: Configured ChargingNetwork.
    """
    network = StochasticNetwork(early_departure=True)
    # network = ChargingNetwork()
    voltage = 415
    evse_type = 'AeroVironment'

    # Define the sets of EVSEs in the Caltech ACN.
    AB_ids = ['AB-{0}'.format(i) for i in range(evse_per_phase)]
    BC_ids = ['BC-{0}'.format(i) for i in range(evse_per_phase)]
    CA_ids = ['CA-{0}'.format(i) for i in range(evse_per_phase)]

    # print(AB_ids)

    # Add Caltech EVSEs
    for evse_id in AB_ids:
        network.register_evse(acnsim.get_evse_by_type(evse_id, evse_type), voltage, 30)
    for evse_id in BC_ids:
        network.register_evse(acnsim.get_evse_by_type(evse_id, evse_type), voltage, -90)
    for evse_id in CA_ids:
        network.register_evse(acnsim.get_evse_by_type(evse_id, evse_type), voltage, 150)

    # Add Caltech Constraint Set
    AB = acnsim.Current(AB_ids)
    BC = acnsim.Current(BC_ids)
    CA = acnsim.Current(CA_ids)

    # Define intermediate currents
    I3a = AB - CA
    I3b = BC - AB
    I3c = CA - BC
    I2a = (1 / 4) * (I3a - I3c)
    I2b = (1 / 4) * (I3b - I3a)
    I2c = (1 / 4) * (I3c - I3b)

    # Build constraint set
    primary_side_constr = transformer_cap * 1000 / 3 / 415
    print(primary_side_constr)
    secondary_side_constr = transformer_cap * 1000 / 3 / 230
    print(secondary_side_constr)
    network.add_constraint(I3a, secondary_side_constr, name='Secondary A')
    network.add_constraint(I3b, secondary_side_constr, name='Secondary B')
    network.add_constraint(I3c, secondary_side_constr, name='Secondary C')
    network.add_constraint(I2a, primary_side_constr, name='Primary A')
    network.add_constraint(I2b, primary_side_constr, name='Primary B')
    network.add_constraint(I2c, primary_side_constr, name='Primary C')

    return network

In [18]:
# How long each time discrete time interval in the simulation should be.
PERIOD = 5  # minutes

# Voltage of the network.
VOLTAGE = 415  # volts

# Default maximum charging rate for each EV battery.
DEFAULT_BATTERY_POWER = 11 # kW

In [19]:
level2_ev_fleet_network = ev_fleet_level_2_network(transformer_cap=30, evse_per_phase=2)

24.096385542168676
43.47826086956522


In [20]:
class CustomUnpicklerJPLdata(pickle.Unpickler):
    def find_class(self, module, name):
        if name == "sklearn.mixture.gaussian_mixture":
            return sklearn.mixture.GaussianMixture
        if name == "GaussianMixture":
            return sklearn.mixture.GaussianMixture
        return super().find_class(module, name)
    
def get_synth_events(sessions_per_day):
    gmm = CustomUnpicklerJPLdata(open('./data/jpl_weekday_40.pkl', "rb")).load()


    # Generate a list of the number of sessions to draw for each day.
    # This generates 30 days of charging demands.
    # num_evs = [0]*2 + [sessions_per_day]*5 + [0]*2 + [sessions_per_day]*5 + [0]*2 + \
    #           [sessions_per_day]*5 + [0]*2 + [sessions_per_day]*5 + [0]*2

    # Generate sessions for 1 day (weekdays only)
    num_evs = [0]*2 + [sessions_per_day]*1

    # Note that because we are drawing from a distribution, some sessions will be
    # invalid, we ignore these sessions and remove the corresponding plugin events.
    gen = GaussianMixtureEvents(pretrained_model=gmm, duration_min=0.08334)

    synth_events = gen.generate_events(num_evs, PERIOD, VOLTAGE, DEFAULT_BATTERY_POWER)
    return synth_events

In [22]:
# Events with 32 EVs per weekday
sessions_32 = get_synth_events(32)

In [23]:
session_copy = deepcopy(sessions_32)
session_list = list(session_copy.queue)
for info in session_list:
    print(info[0], info[1].ev.departure, info[1].ev.requested_energy)

644 711 8.499643652646583
646 766 11.360885114532872
650 675 7.029736184534809
648 775 8.679841916964092
656 753 46.549394044076365
653 783 36.023535705651234
652 771 15.29201493323318
649 757 5.744863539217946
648 743 12.371255289061784
679 774 20.267064068619717
657 767 9.08537804870616
660 764 10.634168134389913
658 772 39.93548469197262
661 789 6.8017804851592905
657 774 12.283281616818956
657 792 9.821179670757466
671 796 7.3407235674864
650 704 5.66477878028465
746 766 7.14494817337594
780 803 11.491058742222723
765 790 10.097169915611715
733 761 13.961163243964819
660 758 6.9846634066818565
663 785 44.55759558329492
669 787 9.96992863753652
663 736 34.05823392815528
777 813 6.498154831455501
730 778 10.049212882806032
717 757 0.8196326140546653
673 792 11.464540716826
666 764 7.4905506219928215
721 766 9.252216405092508


In [25]:
def run_experiment(network, algorithm, events):
    """ Run simulation for the events defined previously and the specified
        network / algorithm / events.
    """
    # Timezone of the ACN we are using.
    # timezone = pytz.timezone('America/Los_Angeles')
    timezone = pytz.timezone('Australia/Melbourne')

    # Start and End times are used when collecting data.
    # start = timezone.localize(datetime(2019, 6, 1))
    # end = timezone.localize(datetime(2019, 7, 1))

    start = timezone.localize(datetime(2023, 9, 13))
    end = timezone.localize(datetime(2023, 9, 14))

    sch = deepcopy(algorithm)
    cn = deepcopy(network)
    signals = {'tariff': TimeOfUseTariff('sce_tou_ev_4_march_2019')}

    sim = acnsim.Simulator(cn, sch, events, start, period=PERIOD, verbose=False, signals=signals)
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
    sim.run()

    r = {'proportion_of_energy_delivered': acnsim.proportion_of_energy_delivered(sim),
         'energy_delivered': sum(ev.energy_delivered for ev in sim.ev_history.values()),
         'num_swaps': cn.swaps,
         'num_never_charged': cn.never_charged,
         'energy_cost': acnsim.energy_cost(sim),
         'demand_charge': acnsim.demand_charge(sim)
         }
    r['total_cost'] = r['energy_cost'] + r['demand_charge']
    r['$/kWh'] = r['total_cost'] / r['energy_delivered']
    return r

In [26]:
uncontrolled = algorithms.UncontrolledCharging()
llf = algorithms.SortedSchedulingAlgo(algorithms.least_laxity_first)

In [27]:
cost_min_obj = [modified_adacharge.ObjectiveComponent(modified_adacharge.total_energy, 1000),
                modified_adacharge.ObjectiveComponent(modified_adacharge.tou_energy_cost),
                modified_adacharge.ObjectiveComponent(modified_adacharge.quick_charge, 1e-5),
                modified_adacharge.ObjectiveComponent(modified_adacharge.equal_share, 1e-12)
               ]
# peak limit: total aggregated current limit 
# cost_min = adacharge.AdaptiveSchedulingAlgorithm(cost_min_obj, solver="MOSEK", quantize=True, reallocate=True, peak_limit=1000, max_recompute=1)
cost_min = modified_adacharge.AdaptiveSchedulingAlgorithm(cost_min_obj, solver="MOSEK", quantize=True, reallocate=False, peak_limit=300, max_recompute=1)

In [30]:
level2_30kW_untrl_32 = run_experiment(level2_ev_fleet_network, uncontrolled, deepcopy(sessions_32))

c:\Users\s3955218\Anaconda3\envs\acnsim\lib\site-packages\acnportal\acnsim\simulator.py:277: UserWarning: Invalid schedule provided at iteration 646. Max violation is 11.947354972638855 A on Secondary A at time index 0.
  warnings.warn(
c:\Users\s3955218\Anaconda3\envs\acnsim\lib\site-packages\acnportal\acnsim\simulator.py:277: UserWarning: Invalid schedule provided at iteration 647. Max violation is 11.947354972638855 A on Secondary A at time index 0.
  warnings.warn(
c:\Users\s3955218\Anaconda3\envs\acnsim\lib\site-packages\acnportal\acnsim\simulator.py:277: UserWarning: Invalid schedule provided at iteration 648. Max violation is 41.18577108450168 A on Secondary B at time index 0.
  warnings.warn(
c:\Users\s3955218\Anaconda3\envs\acnsim\lib\site-packages\acnportal\acnsim\simulator.py:277: UserWarning: Invalid schedule provided at iteration 649. Max violation is 67.37298081484292 A on Secondary B at time index 0.
  warnings.warn(
c:\Users\s3955218\Anaconda3\envs\acnsim\lib\site-packa

In [29]:
level2_30kW_llf_32 = run_experiment(level2_ev_fleet_network, llf, deepcopy(sessions_32))

In [31]:
level2_30kW_cost_min_32 = run_experiment(level2_ev_fleet_network, cost_min, deepcopy(sessions_32))

In [32]:
ev_32 = pd.DataFrame({
    'Level 2: Unctrl: 30 kW : 6 EVSEs':  level2_30kW_untrl_32,
    'Level 2: LLF: 30 kW : 6 EVSEs': level2_30kW_llf_32,
    'Level 2: Min Cost: 30 kW : 6 EVSEs': level2_30kW_cost_min_32
})

In [33]:
ev_32

,Level 2: Unctrl: 30 kW : 6 EVSEs,Level 2: LLF: 30 kW : 6 EVSEs,Level 2: Min Cost: 30 kW : 6 EVSEs
proportion_of_energy_delivered,1.000000,0.443374,0.535591
energy_delivered,457.224079,202.721250,244.885259
num_swaps,22.000000,16.000000,16.000000
num_never_charged,0.000000,10.000000,10.000000
energy_cost,52.383528,35.235762,43.609623
demand_charge,1023.660000,389.456100,476.312100
total_cost,1076.043528,424.691862,519.921723
$/kWh,2.353427,2.094955,2.123124


In [15]:
ev_32.to_csv("results/32_EV_simulation.csv")

In [16]:
pd.read_csv("results/32_EV_simulation.csv", index_col=0)

,Level 2: Unctrl: 30 kW : 6 EVSEs,Level 2: LLF: 30 kW : 6 EVSEs,Level 2: Min Cost: 30 kW : 6 EVSEs
proportion_of_energy_delivered,1.000000,0.422235,0.446793
energy_delivered,399.089403,168.509583,178.310417
num_swaps,22.000000,13.000000,13.000000
num_never_charged,0.000000,13.000000,13.000000
energy_cost,22.440797,9.475294,10.026395
demand_charge,856.152000,361.538100,458.087850
total_cost,878.592797,371.013394,468.114245
$/kWh,2.201494,2.201735,2.625277


In [34]:
level2_35kw_ev_fleet_network = ev_fleet_level_2_network(transformer_cap=35, evse_per_phase=2)

28.112449799196785
50.724637681159415


In [35]:
level2_35kW_untrl_32 = run_experiment(level2_35kw_ev_fleet_network, uncontrolled, deepcopy(sessions_32))

c:\Users\s3955218\Anaconda3\envs\acnsim\lib\site-packages\acnportal\acnsim\simulator.py:277: UserWarning: Invalid schedule provided at iteration 646. Max violation is 4.700978161044652 A on Secondary B at time index 0.
  warnings.warn(
c:\Users\s3955218\Anaconda3\envs\acnsim\lib\site-packages\acnportal\acnsim\simulator.py:277: UserWarning: Invalid schedule provided at iteration 647. Max violation is 4.700978161044652 A on Secondary B at time index 0.
  warnings.warn(
c:\Users\s3955218\Anaconda3\envs\acnsim\lib\site-packages\acnportal\acnsim\simulator.py:277: UserWarning: Invalid schedule provided at iteration 648. Max violation is 33.939394272907485 A on Secondary B at time index 0.
  warnings.warn(
c:\Users\s3955218\Anaconda3\envs\acnsim\lib\site-packages\acnportal\acnsim\simulator.py:277: UserWarning: Invalid schedule provided at iteration 649. Max violation is 60.12660400324874 A on Secondary C at time index 0.
  warnings.warn(
c:\Users\s3955218\Anaconda3\envs\acnsim\lib\site-packag

In [36]:
level2_35kW_llf_32 = run_experiment(level2_35kw_ev_fleet_network, llf, deepcopy(sessions_32))

In [37]:
level2_35kW_cost_min_32 = run_experiment(level2_35kw_ev_fleet_network, cost_min, deepcopy(sessions_32))

In [38]:
ev_32_35kW = pd.DataFrame({
    'Level 2: Unctrl: 35 kW : 6 EVSEs':  level2_35kW_untrl_32,
    'Level 2: LLF: 35 kW : 6 EVSEs': level2_35kW_llf_32,
    'Level 2: Min Cost: 35 kW : 6 EVSEs': level2_35kW_llf_32
})

In [39]:
ev_32_35kW

,Level 2: Unctrl: 35 kW : 6 EVSEs,Level 2: LLF: 35 kW : 6 EVSEs,Level 2: Min Cost: 35 kW : 6 EVSEs
proportion_of_energy_delivered,1.000000,0.468861,0.468861
energy_delivered,457.224079,214.374583,214.374583
num_swaps,22.000000,16.000000,16.000000
num_never_charged,0.000000,10.000000,10.000000
energy_cost,52.383528,38.225440,38.225440
demand_charge,1023.660000,505.315800,505.315800
total_cost,1076.043528,543.541240,543.541240
$/kWh,2.353427,2.535474,2.535474


In [41]:
# Events with 26 EVs per weekday
sessions_26 = get_synth_events(26)

In [42]:
uncontrolled_26 = algorithms.UncontrolledCharging()
llf_26 = algorithms.SortedSchedulingAlgo(algorithms.least_laxity_first)

In [43]:
cost_min_obj_26 = [
                modified_adacharge.ObjectiveComponent(modified_adacharge.total_energy, 1000),
                modified_adacharge.ObjectiveComponent(modified_adacharge.tou_energy_cost),
                modified_adacharge.ObjectiveComponent(modified_adacharge.quick_charge, 1e-5),
                modified_adacharge.ObjectiveComponent(modified_adacharge.equal_share, 1e-12)
               ]
# peak limit: total aggregated current limit 
# cost_min = adacharge.AdaptiveSchedulingAlgorithm(cost_min_obj, solver="MOSEK", quantize=True, reallocate=True, peak_limit=1000, max_recompute=1)
cost_min_26 = modified_adacharge.AdaptiveSchedulingAlgorithm(cost_min_obj_26, solver="MOSEK", quantize=True, reallocate=False, peak_limit=300, max_recompute=1)

In [44]:
level2_35kw_26_ev_fleet_network = ev_fleet_level_2_network(transformer_cap=35, evse_per_phase=2)

28.112449799196785
50.724637681159415


In [45]:
level2_35kW_untrl_26 = run_experiment(level2_35kw_26_ev_fleet_network, uncontrolled, deepcopy(sessions_26))

c:\Users\s3955218\Anaconda3\envs\acnsim\lib\site-packages\acnportal\acnsim\simulator.py:277: UserWarning: Invalid schedule provided at iteration 649. Max violation is 13.275352318840582 A on Secondary A at time index 0.
  warnings.warn(
c:\Users\s3955218\Anaconda3\envs\acnsim\lib\site-packages\acnportal\acnsim\simulator.py:277: UserWarning: Invalid schedule provided at iteration 650. Max violation is 13.275352318840582 A on Secondary A at time index 0.
  warnings.warn(
c:\Users\s3955218\Anaconda3\envs\acnsim\lib\site-packages\acnportal\acnsim\simulator.py:277: UserWarning: Invalid schedule provided at iteration 651. Max violation is 13.275352318840582 A on Secondary A at time index 0.
  warnings.warn(
c:\Users\s3955218\Anaconda3\envs\acnsim\lib\site-packages\acnportal\acnsim\simulator.py:277: UserWarning: Invalid schedule provided at iteration 652. Max violation is 13.275352318840582 A on Secondary A at time index 0.
  warnings.warn(
c:\Users\s3955218\Anaconda3\envs\acnsim\lib\site-pac

In [46]:
level2_35kW_llf_26 = run_experiment(level2_35kw_26_ev_fleet_network, llf, deepcopy(sessions_26))

In [47]:
level2_35kW_cost_min_26 = run_experiment(level2_35kw_26_ev_fleet_network, cost_min, deepcopy(sessions_26))

In [48]:
ev_26_35kW = pd.DataFrame({
    'Level 2: Unctrl: 35 kW : 6 EVSEs':  level2_35kW_untrl_26,
    'Level 2: LLF: 35 kW : 6 EVSEs': level2_35kW_llf_26,
    'Level 2: Min Cost: 35 kW : 6 EVSEs': level2_35kW_cost_min_26
})

In [49]:
ev_26_35kW

,Level 2: Unctrl: 35 kW : 6 EVSEs,Level 2: LLF: 35 kW : 6 EVSEs,Level 2: Min Cost: 35 kW : 6 EVSEs
proportion_of_energy_delivered,1.000000,0.515790,0.572866
energy_delivered,350.517471,180.793333,200.799583
num_swaps,19.000000,10.000000,10.000000
num_never_charged,0.000000,10.000000,10.000000
energy_cost,44.475534,25.454187,29.312360
demand_charge,1023.660000,505.315800,537.499050
total_cost,1068.135534,530.769987,566.811410
$/kWh,3.047310,2.935783,2.822772


In [50]:
cost_min_obj_26_2 = [
                modified_adacharge.ObjectiveComponent(modified_adacharge.total_energy, 1000),
                modified_adacharge.ObjectiveComponent(modified_adacharge.tou_energy_cost),
                modified_adacharge.ObjectiveComponent(modified_adacharge.quick_charge, 1e-5),
                modified_adacharge.ObjectiveComponent(modified_adacharge.equal_share, 1e-11)
               ]
# peak limit: total aggregated current limit 
# cost_min = adacharge.AdaptiveSchedulingAlgorithm(cost_min_obj, solver="MOSEK", quantize=True, reallocate=True, peak_limit=1000, max_recompute=1)
cost_min_26_2 = modified_adacharge.AdaptiveSchedulingAlgorithm(cost_min_obj_26_2, solver="MOSEK", quantize=True, reallocate=True, peak_limit=300, max_recompute=1)

In [51]:
level2_35kW_cost_min_26_2 = run_experiment(level2_35kw_26_ev_fleet_network, cost_min, deepcopy(sessions_26))

In [52]:
ev_26_35kW = pd.DataFrame({
    'Level 2: Unctrl: 35 kW : 6 EVSEs':  level2_35kW_untrl_26,
    'Level 2: LLF: 35 kW : 6 EVSEs': level2_35kW_llf_26,
    'Level 2: Min Cost: 35 kW : 6 EVSEs': level2_35kW_cost_min_26_2
})

In [53]:
ev_26_35kW

,Level 2: Unctrl: 35 kW : 6 EVSEs,Level 2: LLF: 35 kW : 6 EVSEs,Level 2: Min Cost: 35 kW : 6 EVSEs
proportion_of_energy_delivered,1.000000,0.515790,0.559333
energy_delivered,350.517471,180.793333,196.055833
num_swaps,19.000000,10.000000,10.000000
num_never_charged,0.000000,10.000000,10.000000
energy_cost,44.475534,25.454187,28.525712
demand_charge,1023.660000,505.315800,531.062400
total_cost,1068.135534,530.769987,559.588112
$/kWh,3.047310,2.935783,2.854228
